### Imports - Bibliotecas


In [3]:
import pandas as pd
import numpy as np  
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import accuracy_score
from scipy.io import arff
import matplotlib.pyplot as plt
import seaborn as sns

### Carregar dados dos ficheiros auxiliares

In [4]:
# Load the data
data, _ = arff.loadarff('diabetes.arff')
df = pd.DataFrame(data)

df.columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 
              'BMI', 'DiabetesPedigreeFunction', 'Age', 'Class']
df['Class'] = df['Class'].astype(int)

X, y = df.drop('Class', axis=1), df['Class']

### Exercício 1

In [ ]:
# 2. Aplicar ANOVA para identificar os valores F e p
F_values, p_values = f_classif(X, y)

# Criar um DataFrame para armazenar os resultados do ANOVA
anova_results = pd.DataFrame({
    'Feature': X.columns,
    'F-value': F_values,
    'p-value': p_values
})

# Ordenar as variáveis de acordo com o F-value (da maior para a menor)
anova_results_sorted = anova_results.sort_values(by='F-value', ascending=False)

# 3. Identificar as variáveis com melhor e pior poder discriminativo
best_feature = anova_results_sorted['Feature'].iloc[0]
worst_feature = anova_results_sorted['Feature'].iloc[-1]

print(f"\nMelhor variável discriminativa: {best_feature}")
print(f"Pior variável discriminativa: {worst_feature}")

# 4. Função para plotar a densidade de probabilidade condicional por classe
def plot_density_by_class(X, y, feature, feature_name):
    plt.figure(figsize=(10, 6))
    for class_label in np.unique(y):
        subset = X[y == class_label]
        label = "Normal" if class_label == 0 else "Diabetes"
        sns.kdeplot(subset[feature], label=f"{label}", fill=True)
    plt.title(f"Densidade Condicional para {feature_name}")
    plt.xlabel(feature_name)
    plt.ylabel('Densidade')
    plt.legend()
    plt.show()

# 5. Plotar a densidade da variável com o melhor poder discriminativo
plot_density_by_class(X, y, best_feature, f"{best_feature}")

# Plotar a densidade da variável com o pior poder discriminativo
plot_density_by_class(X, y, worst_feature, f"{worst_feature}")

### Exercício 2

In [ ]:
# Convert target labels to numeric if necessary
#y = y.map({'tested_negative': 0, 'tested_positive': 1})

# Stratified 80-20 split with a fixed seed (random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

# Range of min_samples_split values to test
min_samples_split_values = [2, 5, 10, 20, 30, 50, 100]

# Store accuracies for each value of min_samples_split
train_accuracies = []
test_accuracies = []

# Loop through different values of min_samples_split
for min_samples_split in min_samples_split_values:
    # Create a decision tree classifier with the current min_samples_split
    clf = DecisionTreeClassifier(min_samples_split=min_samples_split, random_state=1)
    
    # Train the classifier on the training data
    clf.fit(X_train, y_train)
    
    # Predict on the training data
    y_train_pred = clf.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the test data
    y_test_pred = clf.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)

# Plot the training and testing accuracies
plt.figure(figsize=(10, 6))
plt.plot(min_samples_split_values, train_accuracies, label='Training Accuracy', marker='o', linestyle='--', color='blue')
plt.plot(min_samples_split_values, test_accuracies, label='Testing Accuracy', marker='o', linestyle='-', color='green')
plt.title('Training and Testing Accuracies for Different min_samples_split Values')
plt.xlabel('min_samples_split')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('decision_tree_accuracies.png')  # Save the plot as an image
plt.show()  # Display the plot
plt.close()  # Close the plot

### Exercício 3

Em primeiro lugar, iremos analisar a acurácia nos dados de treino.
A acurácia nestes dados começa extremamente alta (~1.0) quando o valor de min_samples_split é muito baixo.
À medida que o valor de min_samples_split aumenta, a acurácia de treino diminui gradualmente.
Este fenómeno ocorre visto que um valor baixo de min_samples_split permite que a árvore seja muito complexa, resultando em overfitting. Com um min_samples_split maior, a árvore é forçada a generalizar melhor, levando a uma menor acurácia no treino.

Em segundo lugar, iremos analisar a acurácia nos dados de teste.
Inicialmente, a acurácia de teste é baixa (~0.70), mas melhora conforme o valor de min_samples_split aumenta até cerca de 30.
A acurácia de teste atinge um pico em torno de min_samples_split = 50, indicando o ponto de melhor generalização do modelo.
Após esse ponto, a acurácia de teste diminui lentamente, sugerindo que um aumento maior no parâmetro simplifica em demasia o modelo, reduzindo a sua capacidade de capturar as relações nos dados.

Em suma, conseguimos observar que para valores muito baixos, o modelo é altamente complexo e sofre de overfitting, o que indica uma baixa capacidade de generalização. Em contrapartida, para valores muito altos de min_samples_split, o modelo revela-se muito simples e não consegue capturar adequadamente a estrutura dos dados, resultando em underfitting.

### Exercício 4

In [ ]:
# Convert target labels to numeric
#y = y.map({'tested_negative': 0, 'tested_positive': 1})

# Train a Decision Tree with max_depth=3
clf = DecisionTreeClassifier(max_depth=3, random_state=1)
clf.fit(X, y)

# Plot the Decision Tree
plt.figure(figsize=(12, 8))
plot_tree(clf, feature_names=X.columns, class_names=['Normal', 'Diabetes'], filled=True, rounded=True)
plt.title("Decision Tree (max_depth=3)")
plt.savefig('decision_tree_depth3.png')  # Save the plot as an image
plt.show()  # Display the plot

Glucose > 127.5

O primeiro nível de divisão da árvore é baseado nos níveis de glucose. Se a glucose for superior a 127.5, há uma elevada probabilidade de a pessoa ter diabetes.
Após esta divisão, o ramo à direita contém 283 amostras, das quais 174 são classificadas como diabetes (61,5%).

BMI ≤ 29.95 (dado Glucose > 127.5):

Entre os indivíduos com glucose > 127.5, se o BMI for menor ou igual a 29.95, a probabilidade de terem diabetes é alta.
Este ramo tem 283 amostras, com 109 classificadas como normais e 174 como diabetes (Gini = 0.474).

Glucose > 157.5 (dado Glucose > 127.5 e BMI > 29.95):

Se o BMI for superior a 29.95 e a glucose for maior que 157.5, a probabilidade de diabetes aumenta ainda mais.
Nesta divisão, há 207 amostras, com 150 indivíduos classificados como diabetes (72,5%)

Glucose ≤ 157.5 e BMI > 29.95:

Mesmo quando a glucose está elevada (mas ≤ 157.5) e o BMI também está acima de 29.95, há ainda uma elevada probabilidade de diabetes.
Neste caso, há 92 amostras, com 80 indivíduos classificados como diabetes (87%).

Os principais fatores que indicam diabetes são níveis elevados de glucose (especialmente superiores a 127.5) e BMI superior a 29.95. Níveis elevados de glucose, particularmente acima de 157.5, combinados com um BMI elevado, aumentam a probabilidade de diabetes para mais de 70-80%. Níveis baixos de glucose (≤ 127.5) estão mais associados à classe "normal", o que indica que a glucose é o principal preditor neste modelo.

--------------------------------------

Glucose ≤ 127.5 e Idade ≤ 28.5:

Neste ramo, estamos a falar de pessoas com glucose relativamente baixa (≤ 127.5) e idade jovem (≤ 28.5).
Este grupo tem 485 amostras no total, das quais apenas 94 são classificadas como diabetes (19,4% de probabilidade de diabetes), ou seja, a maioria é classificada como normal.
Idade ≤ 28.5 e BMI ≤ 45.4:

Se a pessoa tem menos de 28.5 anos e um BMI ≤ 45.4, a probabilidade de ser normal é muito alta. Das 271 amostras, 248 são classificadas como normais (Gini = 0.155), o que mostra uma probabilidade bastante reduzida de ter diabetes.
Idade ≤ 28.5 e BMI > 45.4:

Para as pessoas mais jovens (≤ 28.5 anos) com BMI muito elevado (superior a 45.4), a chance de diabetes aumenta, mas este grupo tem apenas 4 amostras, e dessas, 3 são classificadas como diabetes. Este é um subgrupo muito pequeno, por isso, a conclusão não é tão forte aqui.
Idade > 28.5:

Quando a idade é superior a 28.5, o próximo critério é o BMI. Aqui, se o BMI for ≤ 26.35, as pessoas têm uma probabilidade elevada de serem normais (143 normais de um total de 214).
Se o BMI for maior que 26.35, existe uma maior probabilidade de diabetes (71 amostras de um total de 214).


O ramo da idade ajuda a identificar quando uma pessoa provavelmente não tem diabetes, especialmente se for jovem (≤ 28.5 anos) e tiver níveis baixos de glucose (≤ 127.5). A árvore indica que pessoas mais jovens, com baixo nível de glucose, têm uma probabilidade muito reduzida de diabetes. Já a idade, por si só, não parece ser um forte indicador de diabetes, mas sim um fator adicional que, quando combinado com outras características como glucose e BMI, contribui para a previsão.

Portanto, o ramo da idade é importante para classificar pessoas como normais, mas, no que diz respeito a identificar diabetes, os fatores mais críticos continuam a ser glucose e BMI.